In [19]:
using BitemporalPostgres
using LifeInsuranceDataModel, LifeInsuranceProduct
using SearchLight
using SearchLightPostgreSQL
using TimeZones
ENV["SEARCHLIGHT_USERNAME"] = "mf"
ENV["SEARCHLIGHT_PASSWORD"] = "mf"
SearchLight.Configuration.load() |> SearchLight.connect

SearchLight.query("DROP SCHEMA public CASCADE")
SearchLight.query("CREATE SCHEMA public")
LifeInsuranceDataModel.load_model()

┌ Info: DROP SCHEMA public CASCADE
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  drop cascades to 37 other objects
DETAIL:  drop cascades to table schema_migrations
drop cascades to table histories
drop cascades to table versions
drop cascades to table workflows
drop cascades to table testtstzranges
drop cascades to table validityintervals
drop cascades to table testdummycomponents
drop cascades to table testdummycomponentrevisions
drop cascades to table testdummysubcomponents
drop cascades to table testdummysubcomponentrevisions
drop cascades to extension btree_gist
drop cascades to function f_bitempranges()
drop cascades to function f_versionrange()
drop cascades to table contractpartnerroles
drop cascades to table tariffitempartnerroles
drop cascades to table tariffitemroles
drop cascades to table productpartroles
drop cascades to table contracts
drop cascades to table contractrevisions
drop cascades

┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/UfvWj/src/LifeInsuranceDataModel.jl:582
┌ Info: CREATE TABLE schema_migrations (version varchar(30))
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: Created table schema_migrations
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:306
┌ Info: CREATE TABLE histories (id BIGSERIAL PRIMARY KEY, dummy INTEGER )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: CREATE TABLE versions (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE workflows (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, tsw_validfrom TIMESTAMPTZ (3), tsdb_validfrom TIMESTAMPTZ (3), is_committed BIGINT , type_of_entity VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: CREATE TABLE testtstzranges (id BIGSERIAL PRIMARY KEY, i TSTZRANGE )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE validityIntervals (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, tsworld_validfrom TIMESTAMPTZ (3), tsworld_invalidfrom TIMESTAMPTZ (3), tsdb_validfrom TIMESTAMPTZ (3), tsdb_invalidfrom TIMESTAMPTZ (3), tsrworld TSTZRANGE , tsrdb TSTZRANGE , is_committed BIGINT )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE testdummyComponents (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/Search

┌ Info: CREATE EXTENSION IF NOT EXISTS btree_gist;
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: ALTER TABLE validityIntervals 
│ ADD CONSTRAINT bitemp EXCLUDE USING GIST (ref_version WITH =, is_committed WITH =, tsrworld WITH &&, tsrdb WITH &&)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


...

┌ Info: CREATE TABLE productItemRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES productitems(id) ON DELETE CASCADE, ref_product BIGINT REFERENCES products(id) ON DELETE CASCADE, ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELETE SET DEFAULT, ref_valid INT8RANGE , description VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TRIGGER pi_versions_trig
│ BEFORE INSERT OR UPDATE ON productitemRevisions
│ FOR EACH ROW EXECUTE PROCEDURE f_versionrange();
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: ALTER TABLE productitemRevisions
│ ADD CONSTRAINT pi_versionrange EXCLUDE USING GIST (ref_component WITH =, ref_valid WITH &&)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE partners (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE partnerRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES partners(id) ON DELETE CASCADE, ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELETE SET DEFAULT, ref_valid INT8RANGE , description VARCHAR , date_of_birth DATE , sex VARCHAR , smoker BOOLEAN )
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/Searc

┌ Info: ALTER TABLE partnerRevisions
│ ADD CONSTRAINT pr_versionrange EXCLUDE USING GIST (ref_component WITH =, ref_valid WITH &&)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE contractPartnerRefs (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_super BIGINT REFERENCES contracts(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE contractPartnerRefRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES contractPartnerRefs(id) ON DELETE CASCADE, ref_role BIGINT REFERENCES contractpartnerroles(id) ON DELETE CASCADE, ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELETE SET DEFAULT, 

┌ Info: ALTER TABLE TariffItemRevisions
│ ADD CONSTRAINT pitrr_versionrange EXCLUDE USING GIST (ref_component WITH =, ref_valid WITH &&)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE TariffItemPartnerRefs (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_super BIGINT REFERENCES tariffitems(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: CREATE TABLE TariffItemPartnerRefRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES TariffItemPartnerRefs(id) ON DELETE CASCADE, ref_role BIGINT REFERENCES tariffitempartnerroles(id) ON DELETE CASCADE, ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 2^53 - 1 REFERENCES versions(id) ON DELE

1×4 Matrix{ProductPartRole}:
 ProductPartRole
| KEY            | VALUE                |
|----------------|----------------------|
| domain::String | ProductPart          |
| id::DbId       | 1                    |
| value::String  | Main Coverage - Life |
  …  ProductPartRole
| KEY            | VALUE                |
|----------------|----------------------|
| domain::String | ProductPart          |
| id::DbId       | 4                    |
| value::String  | Profit participation |


Creating Tariffs

In [20]:
using LifeInsuranceDataModel, LifeInsuranceProduct, SearchLight
import SearchLight: Serializer.serialize

tif = get_tariff_interface(Val(1))
ProfitParticipationTariff = create_tariff("Profit participation", 1, serialize(tif.parameters),serialize(tif.contract_attributes))
tif = get_tariff_interface(Val(2))
PensionTariff = create_tariff("Pension Insurance", 2, serialize(tif.parameters), serialize(tif.contract_attributes))
tif = get_tariff_interface(Val(3))
SingleLifeRiskTariff = create_tariff("Single Life Risk Insurance", 3, serialize(tif.parameters), serialize(tif.contract_attributes))
tif = get_tariff_interface(Val(4))
JointLifeRiskTariff = create_tariff("Joint Life Risk Insurance", 4, serialize(tif.parameters), serialize(tif.contract_attributes),[1,2])



┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 1 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 1 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

NOTICE:  NEW: (1,1,1,"2014-05-30 21:00:01.001+00","2038-01-19 03:14:06.999+00","2023-04-11 17:01:03.866+00","2038-01-19 03:14:06.999+00","[""2014-05-30 21:00:01.001+00"",""2038-01-19 03:14:06.999+00"")","[""2023-04-11 17:01:03.841+00"",""2038-01-19 03:14:06.999+00"")",1)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE "id" = 1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

┌ Info: INSERT INTO tariffrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "description", "interface_id", "parameters", "contract_attributes" ) VALUES ( E'2', E'2', E'9007199254740991', E'Pension Insurance', 2, E'{"interest rate":0.02,"mortality_tables":{"f":{"nonsmoker":"1986-92 CIA – Female Nonsmoker, ANB","smoker":"1986-92 CIA – Female Smoker, ANB"},"m":{"nonsmoker":"1986-92 CIA – Male Nonsmoker, ANB","smoker":"1986-92 CIA – Male Smoker, ANB"}}}', E'{"begin":{"default":"2020-01-01","value":null,"type":"Date"},"net premium":{"default":0,"value":null,"type":"Int"},"m":{"default":0,"value":null,"type":"Int"},"pension rate":{"default":0,"value":null,"type":"Int"},"frequency":{"default":0,"value":null,"type":"Int"},"n":{"default":0,"value":null,"type":"Int"}}' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
NOTICE:  NEW: (2,2,2,9007199254740991,,"Pension Insurance",2,"{""interest rate"

┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 3 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'3' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 3 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsd

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'3' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2014-05-31T21:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: UPDATE validity

┌ Info: INSERT INTO tariffpartnerroles ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'4', NULL, E'4' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffpartnerroles"."id" AS "tariffpartnerroles_id", "tariffpartnerroles"."ref_history" AS "tariffpartnerroles_ref_history", "tariffpartnerroles"."ref_version" AS "tariffpartnerroles_ref_version", "tariffpartnerroles"."ref_super" AS "tariffpartnerroles_ref_super" FROM "tariffpartnerroles" WHERE "id" = 4 ORDER BY tariffpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO tariffpartnerrolerevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "ref_role" ) VALUES ( E'4', E'4', E'9007199254740991', E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchL

4

In [21]:
cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


┌ Info: SELECT "contractpartnerroles"."id" AS "contractpartnerroles_id", "contractpartnerroles"."domain" AS "contractpartnerroles_domain", "contractpartnerroles"."value" AS "contractpartnerroles_value" FROM "contractpartnerroles" ORDER BY contractpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffitempartnerroles"."id" AS "tariffitempartnerroles_id", "tariffitempartnerroles"."domain" AS "tariffitempartnerroles_domain", "tariffitempartnerroles"."value" AS "tariffitempartnerroles_value" FROM "tariffitempartnerroles" ORDER BY tariffitempartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemroles"."value" AS "tariffitemroles_value" FROM "tariffitemroles" ORDER BY tariffitemroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain", "productpartroles"."value" AS "productpartroles_value" FROM "productpartroles" ORDER BY productpartroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


4-element Vector{Int64}:
 1
 2
 3
 4

Pension Insurance Product

In [22]:

pensionInsurance = Product()
pr = ProductRevision(description="Pension")

pp = ProductPart()
ppr = ProductPartRevision(
    ref_tariff=PensionTariff,
    ref_role=ppRole["Main Coverage - Life"],
    description="Main Coverage - Life",
)

pp2 = ProductPart()
ppr2 = ProductPartRevision(
    ref_tariff=ProfitParticipationTariff,
    ref_role=ppRole["Profit participation"],
    description="Profit participation Life Risk",
)

w0 = Workflow(
    type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w0)
create_component!(pensionInsurance, pr, w0)
create_subcomponent!(pensionInsurance, pp, ppr, w0)
create_subcomponent!(pensionInsurance, pp2, ppr2, w0)
commit_workflow!(w0)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 5 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'5' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 5 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'5' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2023-04-02T21:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: UPDATE validity

Single Life Risk Insurance Product

In [23]:

singlelifeRiskInsurance = Product()
pr = ProductRevision(description="Single Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(
    ref_tariff=SingleLifeRiskTariff,
    ref_role=ppRole["Main Coverage - Life"],
    description="Main Coverage - Life",
)

pp2 = ProductPart()
ppr2 = ProductPartRevision(
    ref_tariff=ProfitParticipationTariff,
    ref_role=ppRole["Profit participation"],
    description="Profit participation Life Risk",
)

w0 = Workflow(
    type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w0)
create_component!(singlelifeRiskInsurance, pr, w0)
create_subcomponent!(singlelifeRiskInsurance, pp, ppr, w0)
create_subcomponent!(singlelifeRiskInsurance, pp2, ppr2, w0)
commit_workflow!(w0)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 6 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'6' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 6 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'6' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2023-04-02T21:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: UPDATE validity

A Joint Life Risk Insurance Product

In [24]:

jointlifeRiskInsurance = Product()
pr = ProductRevision(description="Joint Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(
    ref_tariff=JointLifeRiskTariff,
    ref_role=ppRole["Main Coverage - Life"],
    description="Main Coverage - Life",
)

pp2 = ProductPart()
ppr2 = ProductPartRevision(
    ref_tariff=ProfitParticipationTariff,
    ref_role=ppRole["Profit participation"],
    description="Profit participation Life Risk",
)

w0 = Workflow(
    type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w0)
create_component!(jointlifeRiskInsurance, pr, w0)
create_subcomponent!(jointlifeRiskInsurance, pp, ppr, w0)
create_subcomponent!(jointlifeRiskInsurance, pp2, ppr2, w0)
commit_workflow!(w0)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 7 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'7' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 7 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed", "type_of_entity" ) VALUES ( E'7', E'7', E'2038-01-19T03:14:06.999+00:00', E'2023-04-01T21:00:01.001+00:00', 0, E'Product' ) RETURNING id
└ @ S

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'7' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2023-04-02T21:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: UPDATE validity

In [25]:
prs = prsection(1, now(tz"UTC"), ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"), 0)
rolesTariffItemPartner = load_role(TariffItemPartnerRole)

┌ Info: already connected
└ @ LifeInsuranceDataModel /home/mf/dev/julia_depot/packages/LifeInsuranceDataModel/UfvWj/src/LifeInsuranceDataModel.jl:582
┌ Info: SELECT "products"."id" AS "products_id", "products"."ref_history" AS "products_ref_history", "products"."ref_version" AS "products_ref_version" FROM "products" WHERE id=E'1' ORDER BY products.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_inval

┌ Info: SELECT "tariffpartnerroles"."id" AS "tariffpartnerroles_id", "tariffpartnerroles"."ref_history" AS "tariffpartnerroles_ref_history", "tariffpartnerroles"."ref_version" AS "tariffpartnerroles_ref_version", "tariffpartnerroles"."ref_super" AS "tariffpartnerroles_ref_super" FROM "tariffpartnerroles" WHERE ref_super=E'1' ORDER BY tariffpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "tariffpartnerrolerevisions"."id" AS "tariffpartnerrolerevisions_id", "tariffpartnerrolerevisions"."ref_component" AS "tariffpartnerrolerevisions_ref_component", "tariffpartnerrolerevisions"."ref_validfrom" AS "tariffpartnerrolerevisions_ref_validfrom", "tariffpartnerrolerevisions"."ref_invalidfrom" AS "tariffpartnerrolerevisions_ref_invalidfrom", "tariffpartnerrolerevisions"."ref_role" AS "tariffpartnerrolerevisions_ref_role" FROM "tariffpartnerrolerevisions" WHERE ref_component=E'1' and ref_vali

2-element Vector{Dict{String, Any}}:
 Dict("label" => "Insured Person", "value" => 1)
 Dict("label" => "2nd Insured Person", "value" => 2)

create partner woman1 nonsmoker

In [26]:
woman1 = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="woman 1",sex="f",smoker=false)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(woman1, pr, w)
commit_workflow!(w)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 8 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'8' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 8 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


create partner woman2 smoker

In [27]:
woman2 = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="woman 2",sex="f",smoker=true)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(woman2, pr, w)
commit_workflow!(w)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 9 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 9 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/p

Excessive output truncated after 524878 bytes.

┌ Info: INSERT INTO partners ( "ref_history", "ref_version" ) VALUES ( E'9', E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135
┌ Info: SELECT "partners"."id" AS "partners_id", "partners"."ref_history" AS "partners_ref_history", "partners"."ref_version" AS "partners_ref_version" FROM "partners" WHERE "id" = 2 ORDER BY partners.id ASC
└ @ SearchLightPostgreSQL /home/mf/dev/julia_depot/packages/SearchLightPostgreSQL/8KdWH/src/SearchLightPostgreSQL.jl:135


create partner man1 nonsmoker

In [28]:
man1 = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="man 1",sex="m",smoker=false)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(man1, pr, w)
commit_workflow!(w)
man1

Partner
| KEY               | VALUE |
|-------------------|-------|
| id::DbId          | 3     |
| ref_history::DbId | 10    |
| ref_version::DbId | 10    |


create partner man2 smoker

In [29]:
man2 = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="man 2",sex="m",smoker=true)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(man2, pr, w)
commit_workflow!(w)

Contract woman nonsmoker pension

In [30]:
w1 = Workflow(
    type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(
    ref_partner=woman1.id,
    ref_role=cpRole["Policy Holder"],
    description="policiyholder ref properties",
)
create_subcomponent!(c, cpr, cprr, w1)
# pi 1

PartnerRole = tiprRole["Insured Person"]
PartnerroleMap = Dict{Integer,PartnerSection}()
PartnerroleMap[PartnerRole] = psection(woman1.id.value, now(tz"UTC"), w1.tsw_validfrom, 0)

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(
    ref_product=pensionInsurance.id,
    description="from contract creation",
)
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(
    w1,
    cpi,
    pensionInsurance.id.value,
    PartnerroleMap
)
commit_workflow!(w1)

Contract man smoker woman nonsmoker joint life

In [31]:
w1 = Workflow(
    type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(
    ref_partner=woman2.id,
    ref_role=cpRole["Policy Holder"],
    description="policyholder ref properties",
)

create_subcomponent!(c, cpr, cprr, w1)
# pi 1
PartnerroleMap = Dict{Integer,PartnerSection}()
PartnerRole = tiprRole["Insured Person"]
PartnerroleMap[PartnerRole] = psection(woman1.id.value, now(tz"UTC"), w1.tsw_validfrom, 0)
PartnerRole = tiprRole["2nd Insured Person"]
PartnerroleMap[PartnerRole] = psection(man1.id.value, now(tz"UTC"), w1.tsw_validfrom, 0)


cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(
    ref_product=jointlifeRiskInsurance.id.value,
    description="from contract creation",
)
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(
    w1,
    cpi,
    jointlifeRiskInsurance.id.value,
    PartnerroleMap,
)
commit_workflow!(w1)

contract man smoker single life risk

In [32]:
w1 = Workflow(
    type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(
    ref_partner=man2.id,
    ref_role=cpRole["Policy Holder"],
    description="policiyholder ref properties",
)
create_subcomponent!(c, cpr, cprr, w1)
# pi 1

PartnerRole = tiprRole["Insured Person"]
PartnerroleMap = Dict{Integer,PartnerSection}()
PartnerroleMap[PartnerRole] = psection(man2.id.value, now(tz"UTC"), w1.tsw_validfrom, 0)

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(
    ref_product=singlelifeRiskInsurance.id,
    description="from contract creation",
)
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(
    w1,
    cpi,
    singlelifeRiskInsurance.id.value,
    PartnerroleMap
)
commit_workflow!(w1)

calculation pension insurance net preminum

In [33]:
cid = 1
h = find(Contract, SQLWhereExpression("id =?", cid))[1].ref_history
vi = find(ValidityInterval, SQLWhereExpression("ref_history=?", h), order=["ValidityInterval.id"])[1];
txntime = vi.tsdb_validfrom
reftime = vi.tsworld_validfrom

cs = csection(cid, txntime, reftime)

ti = cs.product_items[1].tariff_items[1]


TariffItemSection(TariffItemTariffReference(TariffItemRevision
| KEY                         | VALUE                                                                                                                                                                                                                                                                                                                  |
|-----------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| contract_attributes::String | {"begin":{"default":"2020-01-01","value":null,"type":"Date"},"net premium":{"default":0,"value":null,"type":"Int"},"m":{"default":0,"value":null,"type":"Int"},"pension rate":{"default":0,"value":null,"type":"Int"},"

In [34]:
using JSON, LifeInsuranceProduct,MortalityTables,LifeContingencies,Yields
import LifeContingencies: ä, A
tariffparms = JSON.parse(ti.tariff_ref.ref.revision.parameters)
mts = tariffparms["mortality_tables"]
interface_id = ti.tariff_ref.ref.revision.interface_id
parms = get_tariff_interface(Val(interface_id)).calls
args=parms["calculation_target"]
args["selected"] = "net premium"
args["net premium"]["pension rate"]["value"] = "500"
args["net premium"]["m"]["value"] = "10"
args["net premium"]["n"]["value"] = "20"
args["net premium"]["frequency"]["value"] = "2"
args["net premium"]["begin"]["value"] = string(Date("2023-04-01"))
parms
parms["calculation_target"]
calculate!(Val(interface_id),ti,parms)
parms["result"]

Dict{String, Any} with 1 entry:
  "value" => 6740.85

In [35]:
tariffparms = JSON.parse(ti.tariff_ref.ref.revision.parameters)
println(tariffparms)
interface_id = ti.tariff_ref.ref.revision.interface_id
mts = tariffparms["mortality_tables"]
i = tariffparms["interest rate"]
tgt = get_tariff_interface(Val(interface_id)).calls["calculation_target"]
tgt["selected"] = "net premium"
parms = tgt[tgt["selected"]]
parms["m"] = 10
parms["n"] = 20
parms["frequency"] = 2
parms["begin"] = string(Date("2023-04-01"))


# accessiong partner LifeInsuranceDataModel
dob = ti.partner_refs[1].ref.revision.date_of_birth
smoker = ti.partner_refs[1].ref.revision.smoker ? "smoker" : "nonsmoker"
sex = ti.partner_refs[1].ref.revision.sex
# accessing tariff data

issue_age = TariffUtilities.insurance_age(dob, Date(parms["begin"]))

life = SingleLife(
    mortality=MortalityTables.table(mts[sex][smoker]).select[issue_age])

yield = Yields.Constant(i)      # Using a flat

lc = LifeContingency(life,yield)  # LifeContingenc


r1 = ä(lc)
r2 = ä(lc,1)
r3 = ä(lc, 2)
r4 = ä(lc, 30)

println("r1")
println(r1)
println("r2")
println(r2)
println("r3")
println(r3)
println("r4")
println(r4)
r4

UndefVarError: UndefVarError: TariffUtilities not defined

In [36]:
cid = 2
h = find(Contract, SQLWhereExpression("id =?", cid))[1].ref_history
vi = find(ValidityInterval, SQLWhereExpression("ref_history=?", h), order=["ValidityInterval.id"])[1];
txntime = vi.tsdb_validfrom
reftime = vi.tsworld_validfrom

cs = csection(cid, txntime, reftime)

ti = cs.product_items[1].tariff_items[1]

tariffparms = JSON.parse(ti.tariff_ref.ref.revision.parameters)
println(tariffparms)
mts = tariffparms["mortality_tables"]
i = tariffparms["interest rate"]
interface_id = ti.tariff_ref.ref.revision.interface_id
tgt = get_tariff_interface(Val(interface_id)).calls["calculation_target"]
tgt["selected"] = "net premium"
parms = tgt[tgt["selected"]]
parms["m"] = 10
parms["n"] = 20
parms["frequency"] = 2
parms["begin"] = string(Date("2023-04-01"))

# accessiong partner LifeInsuranceDataModel
dob1 = ti.partner_refs[1].ref.revision.date_of_birth
smoker1 = ti.partner_refs[1].ref.revision.smoker ? "smoker" : "nonsmoker"
sex1 = ti.partner_refs[1].ref.revision.sex
issue_age1 = TariffUtilities.insurance_age(dob1, Date(parms["begin"]))

dob2 = ti.partner_refs[2].ref.revision.date_of_birth
smoker2 = ti.partner_refs[2].ref.revision.smoker ? "smoker" : "nonsmoker"
sex2 = ti.partner_refs[2].ref.revision.sex
issue_age2 = TariffUtilities.insurance_age(dob2, Date(parms["begin"]))

# accessing tariff data


life1 = SingleLife(
    mortality=MortalityTables.table(mts[sex1][smoker1]).select[issue_age1])
life2 = SingleLife(
    mortality=MortalityTables.table(mts[sex2][smoker2]).select[issue_age2])
 jl= JointLife(lives=(life1,life2))

yield = Yields.Constant(i)      # Using a flat

lc = LifeContingency(jl, yield)  # LifeContingenc

n=30
c=5000
r1 = A(lc,30)
r2=ä(lc,n)
5000 * A(lc, 30) / ä(lc, 30, frequency=1n)
A(lc,30)#
 #/ ä(lc, 30,frequency=1n)
 i

UndefVarError: UndefVarError: TariffUtilities not defined